# Best Practices for Using Runners

In [1]:
%pip uninstall -y todd_ai
%pip install --no-build-isolation --extra-index-url https://pypi.org/simple .. > /dev/null


Found existing installation: todd-ai 0.4.0
Uninstalling todd-ai-0.4.0:
  Successfully uninstalled todd-ai-0.4.0
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pathlib
import tempfile
import time
from pprint import pprint
from typing import Any, NoReturn, TypedDict

import torch
import torch.nn.functional as F
import torch.utils.data
from torch import nn

import todd
from todd.runners import Memo

[2023-11-23 16:10:26,112 87795:140704521019008][patches.py:7 todd <module>] INFO: `ipdb` is installed. Using it for debugging.
/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


## Preparation

### Models

In [3]:
@todd.ModelRegistry.register_()
class RunnerModel(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self._weight = torch.nn.Parameter(torch.tensor(0.0))

    @property
    def weight(self) -> torch.nn.Parameter:
        return self._weight

    def _forward(self, x: torch.Tensor) -> torch.Tensor:
        return x * self._weight

    def forward(
        self,
        runner: todd.runners.BaseRunner,
        batch,
        memo: Memo,
        *args,
        **kwargs,
    ) -> Memo:
        log: dict[str, Any] | None = memo.get("log")
        y = self._forward(batch["x"])
        loss = F.l1_loss(y, batch["y"])
        memo["loss"] = loss
        if log is not None:
            log["batch"] = str(batch)
            log["weight"] = f"{self._weight.item():.3f}"
            log["loss"] = f"{loss:.3f}"
        return memo

### Datasets

In [4]:
class Sample(TypedDict):
    x: int
    y: int

In [5]:
@todd.DatasetRegistry.register_()
class RunnerDataset(torch.utils.data.Dataset[int]):
    def __init__(self, n: int) -> None:
        self._data = list(range(1, n + 1))

    def __len__(self) -> int:
        return len(self._data)

    def __getitem__(self, index: int) -> Sample:
        x = self._data[index]
        return Sample(x=x, y=x * 2)

In [6]:
class Batch(TypedDict):
    x: torch.Tensor
    y: torch.Tensor

## Validators

In [7]:
config = todd.Config(
    type='Validator',
    name='validator',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree $work_dirs


[2023-11-23 16:10:27,913 87795:140704521019008][base.py:53 todd.Validator.validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp42nx3r3u
└── validator

2 directories, 0 files


In [8]:
config = todd.Config(
    type='Validator',
    name='validator',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
    callbacks=[dict(type='LogCallback', interval=5)],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree $work_dirs


[2023-11-23 16:10:28,245 87795:140704521019008][base.py:53 todd.Validator.validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:28,251 87795:140704521019008][log.py:99 todd.Validator.validator after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2023-11-23 16:10:28,254 87795:140704521019008][log.py:99 todd.Validator.validator after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2023-11-23 16:10:28,257 87795:140704521019008][log.py:99 todd.Validator.validator after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2023-11-23 16:10:28,259 87795:140704521019008][log.py:99 todd.Validator.validator after_run_iter] INFO: Iter [20/20] batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpme50ogqq
└── validator

2 directories, 0 files


## Trainers

### Iteration Based

In [9]:
config = todd.Config(
    type="IterBasedTrainer",
    name="iter_based_trainer",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[dict(type="LogCallback", interval=1)],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config, work_dir=dict(root=work_dirs)
    )
    runner.run()

[2023-11-23 16:10:28,557 87795:140704521019008][base.py:53 todd.IterBasedTrainer.iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:28,559 87795:140704521019008][log.py:99 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [1/8] batch={'x': tensor([ 1, 10]), 'y': tensor([ 2, 20])} weight=0.000 loss=11.000
[2023-11-23 16:10:28,561 87795:140704521019008][log.py:99 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [2/8] batch={'x': tensor([4, 7]), 'y': tensor([ 8, 14])} weight=0.000 loss=11.000
[2023-11-23 16:10:28,563 87795:140704521019008][log.py:99 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [3/8] batch={'x': tensor([3, 9]), 'y': tensor([ 6, 18])} weight=0.000 loss=12.000
[2023-11-23 16:10:28,565 87795:140704521019008][log.py:99 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [4/8] batch={'x': tensor([6, 8]), 'y': tensor([12, 16])} weight=0.000 loss=14.000
[202

### Epoch Based

In [10]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="epoch_based_trainer",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[dict(type="LogCallback", interval=1)],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config, work_dir=dict(root=work_dirs)
    )
    runner.run()

[2023-11-23 16:10:28,584 87795:140704521019008][base.py:53 todd.EpochBasedTrainer.epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:28,585 87795:140704521019008][log.py:105 todd.EpochBasedTrainer.epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-11-23 16:10:28,587 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [1/15] batch={'x': tensor([6, 1]), 'y': tensor([12,  2])} weight=0.000 loss=7.000
[2023-11-23 16:10:28,588 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [2/15] batch={'x': tensor([3, 8]), 'y': tensor([ 6, 16])} weight=0.000 loss=11.000
[2023-11-23 16:10:28,590 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [3/15] batch={'x': tensor([2, 9]), 'y': tensor([ 4, 18])} weight=0.000 loss=11.000
[2023-11-23 16:10:28,591 87795:140704521019008][log.py:99 todd.Epo

## Callbacks

### Log

In [11]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            collect_env=dict(verbose=False),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config, work_dir=dict(root=work_dirs)
    )
    runner.run()

[2023-11-23 16:10:28,738 87795:140704521019008][log.py:61 todd.Validator.log_callback init] INFO: 
Platform: macOS-14.0
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 0edd804
Git status: M tutorials/runners.ipynb
[2023-11-23 16:10:28,739 87795:140704521019008][base.py:53 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:28,743 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2023-11-23 16:10:28,746 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2023-11-23 16:10:28,748 87795:140704521019008][log.py:99 todd.Validator.

In [12]:
config = todd.Config(
    type='Validator',
    name='log_callback',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
    callbacks=[
        dict(
            type='LogCallback',
            interval=5,
            with_file_handler=True,
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo
    !cat {work_dirs}/log_callback/*.log


[2023-11-23 16:10:28,769 87795:140704521019008][base.py:53 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:28,772 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2023-11-23 16:10:28,774 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2023-11-23 16:10:28,777 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2023-11-23 16:10:28,779 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpcxx6k_vf
└── log_callback
    └── 2023-11-23T16-10-28_769479-08-00.log

2 directories, 1 file

[2023-11-23 16:10:28,769 87795:140704521019008][base.py:53 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:28,772 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2023-11-23 16:10:28,774 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2023-11-23 16:10:28,777 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2023-11-23 16:10:28,779 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] batch={'x':

In [13]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            eta=dict(type="AverageETA"),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config, work_dir=dict(root=work_dirs)
    )
    runner.strategy.module.register_forward_hook(
        lambda *args, **kwargs: time.sleep(0.1)
    )
    runner.run()

[2023-11-23 16:10:29,358 87795:140704521019008][base.py:53 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:29,887 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:01 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2023-11-23 16:10:30,408 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:01 batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2023-11-23 16:10:30,926 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] ETA 0:00:00 batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2023-11-23 16:10:31,442 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] ETA 0:00:00 batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000


In [14]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            eta=dict(type="EMA_ETA", decay=0.2),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config, work_dir=dict(root=work_dirs)
    )
    runner.strategy.module.register_forward_hook(
        lambda *args, **kwargs: time.sleep(0.1 * min(10, runner.iter_))
    )
    runner.run()

[2023-11-23 16:10:31,457 87795:140704521019008][base.py:53 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:32,981 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:03 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2023-11-23 16:10:37,000 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:04 batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2023-11-23 16:10:42,020 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] ETA 0:00:03 batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2023-11-23 16:10:47,043 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] ETA 0:00:00 batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000


In [15]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            collect_env=dict(verbose=False),
            with_file_handler=True,
            eta=dict(type="AverageETA"),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config, work_dir=dict(root=work_dirs)
    )
    runner.run()

[2023-11-23 16:10:47,177 87795:140704521019008][log.py:61 todd.Validator.log_callback init] INFO: 
Platform: macOS-14.0
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 0edd804
Git status: M tutorials/runners.ipynb
[2023-11-23 16:10:47,179 87795:140704521019008][base.py:53 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:47,185 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:00 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2023-11-23 16:10:47,188 87795:140704521019008][log.py:99 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:00 batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2023-11-23 16:10:47,190 87795:140704521019008][l

### Optimize

### Learning Rate Schedule

In [16]:
config = todd.Config(
    type="IterBasedTrainer",
    name="lr_schedule_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScheduleCallback",
            lr_scheduler=dict(type="LinearLR", total_iters=5),
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config, work_dir=dict(root=work_dirs)
    )
    runner.run()

[2023-11-23 16:10:47,209 87795:140704521019008][base.py:53 todd.IterBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:47,213 87795:140704521019008][log.py:99 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([1, 4]), 'y': tensor([2, 8])} weight=0.000 loss=5.000 lr=['1.667e-03']
[2023-11-23 16:10:47,218 87795:140704521019008][log.py:99 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([6, 2]), 'y': tensor([12,  4])} weight=0.004 loss=7.983 lr=['2.333e-03']
[2023-11-23 16:10:47,221 87795:140704521019008][log.py:99 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [3/8] batch={'x': tensor([ 3, 10]), 'y': tensor([ 6, 20])} weight=0.013 loss=12.912 lr=['3.000e-03']
[2023-11-23 16:10:47,223 87795:140704521019008][log.py:99 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [4/8] batch={'x': tensor([7, 

In [17]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="lr_schedule_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=4),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScheduleCallback",
            lr_scheduler=dict(type="LinearLR", total_iters=3),
            by_epoch=True,
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=5,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config, work_dir=dict(root=work_dirs)
    )
    runner.run()

[2023-11-23 16:10:47,246 87795:140704521019008][base.py:53 todd.EpochBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:47,251 87795:140704521019008][log.py:105 todd.EpochBasedTrainer.lr_schedule_callback before_run_epoch] INFO: Epoch [1/5]
[2023-11-23 16:10:47,254 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [1/10] batch={'x': tensor([4, 2]), 'y': tensor([8, 4])} weight=0.000 loss=6.000 lr=['1.667e-03']
[2023-11-23 16:10:47,256 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [2/10] batch={'x': tensor([1, 3]), 'y': tensor([2, 6])} weight=0.005 loss=3.990 lr=['1.667e-03']
[2023-11-23 16:10:47,257 87795:140704521019008][log.py:105 todd.EpochBasedTrainer.lr_schedule_callback before_run_epoch] INFO: Epoch [2/5]
[2023-11-23 16:10:47,260 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.lr_schedule_callback aft

### Learning Rate Scale

In [18]:
config = todd.Config(
    type="IterBasedTrainer",
    name="lr_scale_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScaleCallback",
            lr_scaler=dict(base_batch_size=1),
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config, work_dir=dict(root=work_dirs)
    )
    runner.run()

[2023-11-23 16:10:47,297 87795:140704521019008][lr.py:92 todd.IterBasedTrainer.lr_scale_callback _scale_lr] INFO: base_batch_size=1 batch_size=2 lr_scaler=2.000
[2023-11-23 16:10:47,299 87795:140704521019008][base.py:53 todd.IterBasedTrainer.lr_scale_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:47,303 87795:140704521019008][log.py:99 todd.IterBasedTrainer.lr_scale_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([6, 2]), 'y': tensor([12,  4])} weight=0.000 loss=8.000
[2023-11-23 16:10:47,310 87795:140704521019008][log.py:99 todd.IterBasedTrainer.lr_scale_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([4, 9]), 'y': tensor([ 8, 18])} weight=0.040 loss=12.740
[2023-11-23 16:10:47,322 87795:140704521019008][log.py:99 todd.IterBasedTrainer.lr_scale_callback after_run_iter] INFO: Iter [3/8] batch={'x': tensor([ 7, 10]), 'y': tensor([14, 20])} weight=0.105 loss=16.108
[2023-11-23 16:10:47,326 87795:140704521019008][log.py:9

### Checkpoint

In [19]:
config = todd.Config(
    type="IterBasedTrainer",
    name="checkpoint_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_5 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_5'
    for f in iter_5.glob('*.pth'):
        print(f'{f.name}:')
        pprint(torch.load(f, 'cpu'))
        print()

    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_5),
        )
    runner.run()


[2023-11-23 16:10:47,374 87795:140704521019008][base.py:53 todd.IterBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:47,377 87795:140704521019008][log.py:99 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([2, 5]), 'y': tensor([ 4, 10])} weight=0.000 loss=7.000
[2023-11-23 16:10:47,378 87795:140704521019008][checkpoint.py:72 todd.IterBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp7mep22bd/checkpoint_callback/checkpoints/iter_1
[2023-11-23 16:10:47,387 87795:140704521019008][log.py:99 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([10,  8]), 'y': tensor([20, 16])} weight=0.018 loss=17.843
[2023-11-23 16:10:47,391 87795:140704521019008][checkpoint.py:72 todd.IterBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp7mep22bd
└── checkpoint_callback
    └── checkpoints
        ├── iter_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_4
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_5
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_6
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── opt

[2023-11-23 16:10:47,874 87795:140704521019008][checkpoint.py:46 todd.IterBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp7mep22bd/checkpoint_callback/checkpoints/iter_5
[2023-11-23 16:10:47,877 87795:140704521019008][base.py:56 todd.IterBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2023-11-23 16:10:47,878 87795:140704521019008][base.py:53 todd.IterBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:47,882 87795:140704521019008][log.py:99 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [6/8] batch={'x': tensor([ 7, 10]), 'y': tensor([14, 20])} weight=0.137 loss=15.831
[2023-11-23 16:10:47,882 87795:140704521019008][checkpoint.py:72 todd.IterBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp7mep22bd/checkpoint_callback/checkpoint

strategy.pth:
{}

optim.pth:
{'param_groups': [{'dampening': 0,
                   'differentiable': False,
                   'foreach': None,
                   'lr': 0.005,
                   'maximize': False,
                   'momentum': 0,
                   'nesterov': False,
                   'params': [0],
                   'weight_decay': 0}],
 'state': {0: {'momentum_buffer': None}}}

meta.pth:
{'iter_': 5}

model.pth:
OrderedDict([('_weight', tensor(0.1375))])

callbacks.pth:
{'callbacks': [{}, {}, {}]}



In [20]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="checkpoint_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=2),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_8 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_8'
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_8),
        )
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    iter_10 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_10'
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_10),
        )
    runner.run()


[2023-11-23 16:10:47,933 87795:140704521019008][base.py:53 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:47,935 87795:140704521019008][log.py:105 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [1/3]
[2023-11-23 16:10:47,938 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/15] batch={'x': tensor([5, 2]), 'y': tensor([10,  4])} weight=0.000 loss=7.000
[2023-11-23 16:10:47,940 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/15] batch={'x': tensor([9, 8]), 'y': tensor([18, 16])} weight=0.018 loss=16.851
[2023-11-23 16:10:47,941 87795:140704521019008][checkpoint.py:72 todd.EpochBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpon963xtt/checkpoint_callback/checkpoints/iter_2
[2023-11-23 16:10:47,945 87795:1407


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpon963xtt
└── checkpoint_callback
    └── checkpoints
        ├── iter_10
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_12
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_14
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_4
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_6
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── 

[2023-11-23 16:10:48,437 87795:140704521019008][checkpoint.py:46 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpon963xtt/checkpoint_callback/checkpoints/iter_8
[2023-11-23 16:10:48,440 87795:140704521019008][base.py:56 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2023-11-23 16:10:48,441 87795:140704521019008][base.py:53 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:48,442 87795:140704521019008][log.py:105 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [2/3]
[2023-11-23 16:10:48,445 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [9/15] batch={'x': tensor([10,  9]), 'y': tensor([20, 18])} weight=0.210 loss=17.005
[2023-11-23 16:10:48,447 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.checkpoi


--------------------



[2023-11-23 16:10:48,898 87795:140704521019008][checkpoint.py:46 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpon963xtt/checkpoint_callback/checkpoints/iter_10
[2023-11-23 16:10:48,902 87795:140704521019008][base.py:56 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2023-11-23 16:10:48,903 87795:140704521019008][base.py:53 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:48,904 87795:140704521019008][log.py:105 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [3/3]
[2023-11-23 16:10:48,907 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [11/15] batch={'x': tensor([6, 5]), 'y': tensor([12, 10])} weight=0.278 loss=9.474
[2023-11-23 16:10:48,909 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.checkpoin

In [21]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="checkpoint_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1, by_epoch=True),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    epoch_2 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'epoch_2'
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(epoch_2),
        )
    runner.run()


[2023-11-23 16:10:48,946 87795:140704521019008][base.py:53 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:48,948 87795:140704521019008][log.py:105 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [1/3]
[2023-11-23 16:10:48,951 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/15] batch={'x': tensor([8, 3]), 'y': tensor([16,  6])} weight=0.000 loss=11.000
[2023-11-23 16:10:48,953 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/15] batch={'x': tensor([6, 7]), 'y': tensor([12, 14])} weight=0.027 loss=12.821
[2023-11-23 16:10:48,955 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [3/15] batch={'x': tensor([10,  9]), 'y': tensor([20, 18])} weight=0.060 loss=18.430
[2023-11-23 16:10:48,957 87795:140704521019008][log.py:99 todd.


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpp2w_40pe
└── checkpoint_callback
    └── checkpoints
        ├── epoch_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        └── latest -> epoch_3

7 directories, 15 files



[2023-11-23 16:10:49,417 87795:140704521019008][checkpoint.py:46 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpp2w_40pe/checkpoint_callback/checkpoints/epoch_2
[2023-11-23 16:10:49,421 87795:140704521019008][base.py:56 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2023-11-23 16:10:49,422 87795:140704521019008][base.py:53 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:49,423 87795:140704521019008][log.py:105 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [3/3]
[2023-11-23 16:10:49,425 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [11/15] batch={'x': tensor([9, 8]), 'y': tensor([18, 16])} weight=0.275 loss=14.663
[2023-11-23 16:10:49,427 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.checkpoi

### Monitor

In [22]:
class CustomError(RuntimeError):
    pass

In [23]:
@todd.RunnerRegistry.register_()
class FaultyValidator(todd.runners.Validator):
    def _run_iter(self, *args, **kwargs) -> NoReturn:
        raise CustomError("faulty runner")

In [24]:
config = todd.Config(
    type='FaultyValidator',
    name='monitor_callback',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
    callbacks=[
        dict(type='MonitorCallback'),
        dict(type='LogCallback', interval=5, with_file_handler=True),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    try:
        runner.run()
    except CustomError as e:
        pass

    !echo
    !cat {work_dirs}/monitor_callback/*.log


[2023-11-23 16:10:49,470 87795:140704521019008][base.py:53 todd.FaultyValidator.monitor_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:49,471 87795:140704521019008][monitor.py:26 todd.FaultyValidator.monitor_callback __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader._SingleProcessDataLoaderIter object at 0x122333a50>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 221, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_87795/3179889958.py", line 4, in _run_iter
    raise CustomError("faulty runner")
CustomError: faulty runner



[2023-11-23 16:10:49,470 87795:140704521019008][base.py:53 todd.FaultyValidator.monitor_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:49,471 87795:140704521019008][monitor.py:26 todd.FaultyValidator.monitor_callback __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader._SingleProcessDataLoaderIter object at 0x122333a50>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 221, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_87795/3179889958.py", line 4, in _run_iter
    raise CustomError("faulty runner")
CustomError: faulty runner


### Priorities

## Strategies

In [25]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="strategy_load_model_from",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1, by_epoch=True),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    epoch_2 = (pathlib.Path(work_dirs) / 'strategy_load_model_from' / 'checkpoints' / 'epoch_2' / 'model.pth')
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.strategy.load_model_from(epoch_2)
    runner.run()


[2023-11-23 16:10:49,779 87795:140704521019008][base.py:53 todd.EpochBasedTrainer.strategy_load_model_from __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:49,781 87795:140704521019008][log.py:105 todd.EpochBasedTrainer.strategy_load_model_from before_run_epoch] INFO: Epoch [1/3]
[2023-11-23 16:10:49,785 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [1/15] batch={'x': tensor([7, 1]), 'y': tensor([14,  2])} weight=0.000 loss=8.000
[2023-11-23 16:10:49,787 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [2/15] batch={'x': tensor([5, 3]), 'y': tensor([10,  6])} weight=0.020 loss=7.920
[2023-11-23 16:10:49,789 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [3/15] batch={'x': tensor([ 4, 10]), 'y': tensor([ 8, 20])} weight=0.040 loss=13.720
[2023-11-23 16:10:49,791 87795:140704521


--------------------



[2023-11-23 16:10:50,272 87795:140704521019008][base.py:53 todd.EpochBasedTrainer.strategy_load_model_from __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-11-23 16:10:50,273 87795:140704521019008][base.py:70 todd.EpochBasedTrainer.strategy_load_model_from load_model_from] INFO: Loading model from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpfgawtv3w/strategy_load_model_from/checkpoints/epoch_2/model.pth
[2023-11-23 16:10:50,275 87795:140704521019008][base.py:56 todd.EpochBasedTrainer.strategy_load_model_from load_model_state_dict] INFO: <All keys matched successfully>
[2023-11-23 16:10:50,276 87795:140704521019008][log.py:105 todd.EpochBasedTrainer.strategy_load_model_from before_run_epoch] INFO: Epoch [1/3]
[2023-11-23 16:10:50,279 87795:140704521019008][log.py:99 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [1/15] batch={'x': tensor([8, 5]), 'y': tensor([16, 10])} weight=0.275 loss=11.212
[2023-11-23 16:10:50,281 87795:14070452

## Dry Run

In [26]:
todd.Store.DRY_RUN = True